In [1]:
import numpy as np
import unittest

In [2]:
# Fuzzy TOPSIS implementation
def fuzzy_topsis(decision_matrix, weights, criteria_type):
    """
    Implementation of the Fuzzy TOPSIS method for multicriteria decision analysis.

    Parameters:
    decision_matrix (ndarray): Matrix (m x n x 3) with m alternatives, n criteria, and 3 values representing fuzzy numbers (low, medium, high).
    weights (list or ndarray): Weights for each criterion.
    criteria_type (list): A list specifying whether each criterion should be 'max' or 'min'.

    Returns:
    ndarray: Score for each alternative, ranking them from best to worst.
    """
    # Step 1: Normalize the decision matrix
    norm_matrix = np.zeros_like(decision_matrix, dtype=float)
    for j in range(decision_matrix.shape[1]):
        if criteria_type[j] == 'max':
            max_value = np.max(decision_matrix[:, j, 2])
            norm_matrix[:, j, 0] = decision_matrix[:, j, 0] / max_value
            norm_matrix[:, j, 1] = decision_matrix[:, j, 1] / max_value
            norm_matrix[:, j, 2] = decision_matrix[:, j, 2] / max_value
        else:
            min_value = np.min(decision_matrix[:, j, 0])
            norm_matrix[:, j, 0] = min_value / decision_matrix[:, j, 2]
            norm_matrix[:, j, 1] = min_value / decision_matrix[:, j, 1]
            norm_matrix[:, j, 2] = min_value / decision_matrix[:, j, 0]

    # Step 2: Apply the weights
    weighted_matrix = np.zeros_like(norm_matrix, dtype=float)
    for j in range(norm_matrix.shape[1]):
        weighted_matrix[:, j, 0] = norm_matrix[:, j, 0] * weights[j]
        weighted_matrix[:, j, 1] = norm_matrix[:, j, 1] * weights[j]
        weighted_matrix[:, j, 2] = norm_matrix[:, j, 2] * weights[j]

    # Step 3: Determine the fuzzy positive ideal solution (FPIS) and fuzzy negative ideal solution (FNIS)
    fpis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    fnis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    for j in range(weighted_matrix.shape[1]):
        if criteria_type[j] == 'max':
            fpis[j, 0] = np.max(weighted_matrix[:, j, 0])
            fpis[j, 1] = np.max(weighted_matrix[:, j, 1])
            fpis[j, 2] = np.max(weighted_matrix[:, j, 2])
            fnis[j, 0] = np.min(weighted_matrix[:, j, 0])
            fnis[j, 1] = np.min(weighted_matrix[:, j, 1])
            fnis[j, 2] = np.min(weighted_matrix[:, j, 2])
        else:
            fpis[j, 0] = np.min(weighted_matrix[:, j, 0])
            fpis[j, 1] = np.min(weighted_matrix[:, j, 1])
            fpis[j, 2] = np.min(weighted_matrix[:, j, 2])
            fnis[j, 0] = np.max(weighted_matrix[:, j, 0])
            fnis[j, 1] = np.max(weighted_matrix[:, j, 1])
            fnis[j, 2] = np.max(weighted_matrix[:, j, 2])

    # Step 4: Calculate the distances to FPIS and FNIS
    distance_to_fpis = np.sqrt(np.sum(np.power(weighted_matrix - fpis, 2), axis=(1, 2)))
    distance_to_fnis = np.sqrt(np.sum(np.power(weighted_matrix - fnis, 2), axis=(1, 2)))

    # Step 5: Calculate the similarity to the ideal solution
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity_to_ideal = np.divide(distance_to_fnis, (distance_to_fpis + distance_to_fnis))
        similarity_to_ideal[np.isnan(similarity_to_ideal)] = 0  # Set NaN values to 0 to handle division by zero

    return similarity_to_ideal

In [3]:
# Example usage for Fuzzy TOPSIS
decision_matrix_fuzzy = np.array([[[2, 3, 4], [3, 4, 5], [1, 2, 3]],
                                 [[1, 2, 3], [2, 3, 4], [2, 3, 4]],
                                 [[3, 4, 5], [4, 5, 6], [3, 4, 5]]])
weights = [0.5, 0.3, 0.2]
criteria_type = ['max', 'max', 'min']

fuzzy_scores = fuzzy_topsis(decision_matrix_fuzzy, weights, criteria_type)
fuzzy_ranking = np.argsort(fuzzy_scores)[::-1]  # Get the ranking in descending order

print("Fuzzy TOPSIS Scores:", fuzzy_scores)
print("Fuzzy TOPSIS Ranking (best to worst):", fuzzy_ranking + 1)  # Adding 1 to make it 1-based index for convenience

Fuzzy TOPSIS Scores: [0.4446656  0.21518717 1.        ]
Fuzzy TOPSIS Ranking (best to worst): [3 1 2]


In [4]:
# Unit tests for Fuzzy TOPSIS
class TestFuzzyTOPSIS(unittest.TestCase):
    def setUp(self):
        self.decision_matrix_fuzzy = np.array([[[2, 3, 4], [3, 4, 5], [1, 2, 3]],
                                               [[1, 2, 3], [2, 3, 4], [2, 3, 4]],
                                               [[3, 4, 5], [4, 5, 6], [3, 4, 5]]])
        self.weights = [0.5, 0.3, 0.2]
        self.criteria_type = ['max', 'max', 'min']

    def test_fuzzy_topsis(self):
        fuzzy_scores = fuzzy_topsis(self.decision_matrix_fuzzy, self.weights, self.criteria_type)
        fuzzy_ranking = np.argsort(fuzzy_scores)[::-1]
        # Instead of asserting exact ranking, ensure the ranking makes sense
        self.assertTrue(fuzzy_ranking[0] in [0, 2])  # The best should be either alternative 1 or 3
        self.assertTrue(fuzzy_ranking[-1] in [1])  # The worst should be alternative 2
        self.assertEqual(len(set(fuzzy_ranking)), len(fuzzy_ranking))  # Ensure all alternatives are ranked uniquely

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
